A sentiment classifier that outperforms the current top HuggingFace leaderboard on the `rotten_tomatoes` sentiment dataset.

In [1]:
import sys
import os

notebook_dir = os.path.abspath('')
parent_dir = os.path.dirname(notebook_dir)
sys.path.insert(0, parent_dir)

In [2]:
# Auto reload modules
%load_ext autoreload
%autoreload 2

In [4]:
from dataset import Dataset, Embedder
from model import ClassificationModel
import json

In [5]:
# Load the config file
# type = "sentiment_imdb"
type = "sentiment_rottentomatoes"

with open (f'configs/{type}.json') as f:
    config = json.load(f)

In [6]:
# Create the test and validation datasets

# model = 'E5'
# model = 'openai'
model = 'InstructorXL'

embedder = Embedder(model)
config_dataset = config['datasets'][0]
dataset_train = Dataset(type, config_dataset, split='train')
dataset_test = Dataset(type, config_dataset, split='test')

load INSTRUCTOR_Transformer
max_seq_length  512


Found cached dataset rotten_tomatoes (/root/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46)
Loading cached processed dataset at /root/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46/cache-364b3bd23f693fa4.arrow
Found cached dataset rotten_tomatoes (/root/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46)
Loading cached processed dataset at /root/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46/cache-ab72149132f0faf2.arrow


In [11]:
import torch

device = torch.device('cuda')
embedder.embed(dataset_train, batch_size=50, device=device, save=True)
embedder.embed(dataset_test, batch_size=50, device=device, save=True)

  0%|          | 0/171 [00:00<?, ?it/s]


AttributeError: 'NoneType' object has no attribute 'cpu'

In [38]:
# Fit and test the linear model
config_model = config['model']
model = ClassificationModel(config_model)

result_train = model.fit(dataset_train)
print('Training Results', result_train)

result_test = model.test(dataset_test)
print('Validation Results:', result_test)

Training Results {'accuracy': 0.91616, 'f1': 0.9166401527203308}
Validation Results: {'accuracy': 0.89936, 'f1': 0.8990369181380418}


In [ ]:
path = f'models/{type}/model.npz'
model.save(path)
mode.toTorch(save=True, path=path)